In [9]:
import pandas as pd
import sqlite3

In [4]:
# Load CSVs
day1 = pd.read_csv('products_day1.csv')
day2 = pd.read_csv('products_day2.csv')

In [11]:
# Create SQL DB in memory
conn = sqlite3.connect(':memory:')
day1.to_sql('products_day1', conn, index=False, if_exists='replace')
day2.to_sql('products_day2', conn, index=False, if_exists='replace')

6

In [ ]:
# 2.Find Added or Removed Rows

# Removed from Day 2
removed_query = """
SELECT *, 'REMOVED' as change_type FROM products_day1
EXCEPT
SELECT *, 'REMOVED' as change_type FROM products_day2
"""
removed_df = pd.read_sql_query(removed_query, conn)

# Added in Day 2
added_query = """
SELECT *, 'ADDED' as change_type FROM products_day2
EXCEPT
SELECT *, 'ADDED' as change_type FROM products_day1
"""
added_df = pd.read_sql_query(added_query, conn)

# Combine both for full change set
changes_df = pd.concat([removed_df, added_df], ignore_index=True)
print(changes_df)


   product_id              name     category  price  stock change_type
0         101  Wireless\n Mouse  Electronics  25.99    120     REMOVED
1         102      Water Bottle      Kitchen  10.50    200     REMOVED
2         105         Desk Lamp    Furniture  35.00     90     REMOVED
3         106          Notebook   Stationery   2.99    300     REMOVED
4         101    Wireless Mouse  Electronics  23.99    120       ADDED
5         102      Water Bottle      Kitchen  10.50    180       ADDED
6         105         Desk Lamp    Furniture  37.00     90       ADDED
7         107       Smart Watch    Wearables  99.00     50       ADDED


In [ ]:
# 3.Column-Level Differences

column_diff_query = """
SELECT a.product_id, 'price' AS column_changed, a.price AS old_value, b.price AS new_value
FROM products_day1 a JOIN products_day2 b ON a.product_id = b.product_id
WHERE a.price != b.price

UNION

SELECT a.product_id, 'stock' AS column_changed, a.stock AS old_value, b.stock AS new_value
FROM products_day1 a JOIN products_day2 b ON a.product_id = b.product_id
WHERE a.stock != b.stock;
"""

column_diff_df = pd.read_sql_query(column_diff_query, conn)
print(column_diff_df)

   product_id column_changed  old_value  new_value
0         101          price      25.99      23.99
1         102          stock     200.00     180.00
2         105          price      35.00      37.00
